In [20]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import csv
import requests
import json

In [21]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.maximize_window()

In [ ]:
""" 
Во время сбора данных, WB, естественно банит нас, поэтому была введена переменная error_page - страница на которой был получен бан. 
Мы сохраняем данные, которые мы уже успели получить в файл с этим error page и начинаем скреппинг уже с этой страницы снова где-то через 10 минут
"""
data = []
error_page = 23
headers = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:137.0) Gecko/20100101 Firefox/137.0"
}

for i in range(23 - error_page):
    
    # открываем URL
    url = f"https://www.wildberries.ru/catalog/0/search.aspx?page={i+error_page}&sort=popular&search=%D0%BB%D0%B5%D1%82%D0%BD%D1%8F%D1%8F+%D0%BE%D0%B4%D0%B5%D0%B6%D0%B4%D0%B0"
    driver.get(url)
    time.sleep(7) # Чуть замедлимся, чтобы вб не отказал нам в доступе
    
    # Прокручиваем страницу, чтобы прогрузить все товары
    for i in range(4):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
        
    items = driver.find_elements(By.CLASS_NAME, "product-card")
    for item in items:
        try:
            # Основная информация
            name = item.find_element(By.CSS_SELECTOR, ".product-card__name").text.strip()
            price = item.find_element(By.CSS_SELECTOR, ".price__lower-price").text.strip()
            link = item.find_element(By.CSS_SELECTOR, ".product-card__link").get_attribute("href")
            
            # Извлекаем артикул из ссылки
            articul = link.split('/')[4]
            
            # Формируем URL для истории цен
            vol = articul[:4]
            part = articul[:6]
            
            # Получаем историю цен
            price_history = None
    
            # т.к. у WB есть несколько поддоменов basket-10, basket-20 итд, где может храниться история цен, попробуем взять ее с каждого
            basket_nums = []
            for i in range(23):
                num = i+1
                if num < 10:
                    basket_nums.append(f"0{num}")
                else:
                    basket_nums.append(f"{num}")
            for i in basket_nums:
                price_history_url = f"https://basket-{i}.wbbasket.ru/vol{vol}/part{part}/{articul}/info/price-history.json"
                response = requests.get(price_history_url, headers=headers)
                if response.status_code == 200:
                    price_history = response.json()
                    break  
            
            if price_history != None:
                data.append({
                "Название": name,
                "Артикул": articul,
                "Текущая_цена": price,
                "История_цен": json.dumps(price_history, ensure_ascii=False) if price_history else None
            })
                print(f"{name} {articul} {price}")
    
        except Exception as e:
            print(f"Ошибка: {e}")
            print(f"Задайте error_page = {i+error_page}") # Лучше делать это вручную, так как это получилось стабильнее

In [24]:
error_page = 23
with open(f"../data/wildberries_data_{error_page}.csv", "w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=["Название", "Артикул", "Текущая_цена", "История_цен"])
    writer.writeheader()
    writer.writerows(data)

driver.quit()